In [1]:
import numpy as np
import sklearn as skl
from numpy import genfromtxt
from sklearn import kernel_ridge
import os
from sklearn.model_selection import GridSearchCV
print(len(os.sched_getaffinity(0)))

16


In [2]:
path_to_train = '/home/sathvikc/AMATH-482-2/AMATH-482/Homework Assignments/Homework 3/wine_training.csv'
path_to_test = '/home/sathvikc/AMATH-482-2/AMATH-482/Homework Assignments/Homework 3/wine_test.csv'

train = genfromtxt(path_to_train, delimiter=',')
test = genfromtxt(path_to_test, delimiter=',')

print('TRAIN SET')
print(train)
print('ALL BUT LAST')
print(train[:, :-1])
print('LAST')
print(train[:, -1])

TRAIN SET
[[ 6.1   0.6   0.08 ...  0.54 11.    5.  ]
 [10.3   0.53  0.48 ...  0.59  9.3   6.  ]
 [ 6.9   0.49  0.19 ...  0.64  9.8   6.  ]
 ...
 [ 7.    0.38  0.49 ...  0.77 11.4   6.  ]
 [10.    0.35  0.47 ...  0.52 12.    6.  ]
 [ 7.    0.6   0.3  ...  1.17 10.2   5.  ]]
ALL BUT LAST
[[ 6.1   0.6   0.08 ...  3.38  0.54 11.  ]
 [10.3   0.53  0.48 ...  3.12  0.59  9.3 ]
 [ 6.9   0.49  0.19 ...  3.38  0.64  9.8 ]
 ...
 [ 7.    0.38  0.49 ...  3.39  0.77 11.4 ]
 [10.    0.35  0.47 ...  3.23  0.52 12.  ]
 [ 7.    0.6   0.3  ...  3.3   1.17 10.2 ]]
LAST
[5. 6. 6. ... 6. 6. 5.]


In [3]:
norm_train = (train - train.mean(axis=0)) / train.std(axis=0)
norm_test = (test - test.mean(axis=0)) / test.std(axis=0)

In [4]:
print(f'TRAIN STD PER COLUMN {train.std(axis=0)}')
print(f'TRAIN MEAN PER COLUMN {train.mean(axis=0)}')

TRAIN STD PER COLUMN [1.73802726e+00 1.78924336e-01 1.95148774e-01 1.42389434e+00
 5.11090930e-02 1.06931743e+01 3.14087940e+01 1.87298299e-03
 1.52519210e-01 1.78183748e-01 1.04853788e+00 7.98733136e-01]
TRAIN MEAN PER COLUMN [ 8.35183857  0.52507623  0.27384753  2.55399103  0.08926547 16.15695067
 46.38699552  0.99683448  3.31076233  0.66102242 10.39699552  5.63856502]


In [5]:
#Use linear regression to fit a linear model to the training set 
LinearRegressor = skl.linear_model.LinearRegression()
LinearRegressor.fit(norm_train[:, :-1], norm_train[:, -1])
predicted_train = LinearRegressor.predict(norm_train[:, :-1])
print(f'LINEAR REGRESSION TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, norm_train[:, -1])}')
predicted_test = LinearRegressor.predict(norm_test[:, :-1])
print(f'LINEAR REGRESSION TEST MSE: {skl.metrics.mean_squared_error(predicted_test, norm_test[:, -1])}')

LINEAR REGRESSION TRAINING MSE: 0.6278484956554817
LINEAR REGRESSION TEST MSE: 0.7021527395264012


In [6]:
#Use kernel ridge regression to fit a non-linear model to the training set

#Gaussian (RBF) Kernel 
RBFRegresor = kernel_ridge.KernelRidge(kernel = 'rbf')  

#Laplacian kernel
LaplacianRegresor = kernel_ridge.KernelRidge(kernel = 'laplacian')

In [7]:
# range of values of sigma to try 
K_sgm = 20
K_lmbd = 20

sgm = np.linspace(-4, 4, K_sgm)
lmbd = np.linspace(-4, 4, K_lmbd)

scores = np.zeros((K_sgm, K_lmbd))
scores_std = np.zeros((K_sgm, K_lmbd))

#RBF 
cv = GridSearchCV(estimator = RBFRegresor, param_grid = {
    'alpha': 2 * lmbd,
    'gamma': 1 / (2 ** (sgm**2))
    }, cv = 5, n_jobs=len(os.sched_getaffinity(0)))

cv.fit(norm_train[:, :-1], norm_train[:, -1])

/home/sathvikc/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/sathvikc/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/sathvikc/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/sathvikc/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/sathvi

GridSearchCV(cv=5, estimator=KernelRidge(kernel='rbf'), n_jobs=16,
             param_grid={'alpha': array([-8.        , -7.15789474, -6.31578947, -5.47368421, -4.63157895,
       -3.78947368, -2.94736842, -2.10526316, -1.26315789, -0.42105263,
        0.42105263,  1.26315789,  2.10526316,  2.94736842,  3.78947368,
        4.63157895,  5.47368421,  6.31578947,  7.15789474,  8.        ]),
                         'gamma': array([1.52587891e-05, 1.39363051e-04, 9.95494407e-04, 5.56152131e-03,
       2.43003199e-02, 8.30412689e-02, 2.21942036e-01, 4.63926049e-01,
       7.58440565e-01, 9.69745896e-01, 9.69745896e-01, 7.58440565e-01,
       4.63926049e-01, 2.21942036e-01, 8.30412689e-02, 2.43003199e-02,
       5.56152131e-03, 9.95494407e-04, 1.39363051e-04, 1.52587891e-05])})

In [8]:
best_alpha_rbf = cv.best_params_['alpha']
best_gamma_rbf = cv.best_params_['gamma']
print(best_alpha_rbf)
print(best_gamma_rbf)

0.4210526315789469
0.08304126891083763


In [9]:
# range of values of sigma to try 
K_sgm = 20
K_lmbd = 20

sgm = np.linspace(-4, 4, K_sgm)
lmbd = np.linspace(-4, 4, K_lmbd)

scores = np.zeros((K_sgm, K_lmbd))
scores_std = np.zeros((K_sgm, K_lmbd))

#Laplacian
cv = GridSearchCV(estimator = LaplacianRegresor, param_grid = {
    'alpha': 2 * lmbd,
    'gamma': 1 / (2 ** (sgm))
    }, cv = 5, n_jobs=len(os.sched_getaffinity(0)))

cv.fit(norm_train[:, :-1], norm_train[:, -1])

/home/sathvikc/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/sathvikc/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/sathvikc/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/sathvikc/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/sathvi

GridSearchCV(cv=5, estimator=KernelRidge(kernel='laplacian'), n_jobs=16,
             param_grid={'alpha': array([-8.        , -7.15789474, -6.31578947, -5.47368421, -4.63157895,
       -3.78947368, -2.94736842, -2.10526316, -1.26315789, -0.42105263,
        0.42105263,  1.26315789,  2.10526316,  2.94736842,  3.78947368,
        4.63157895,  5.47368421,  6.31578947,  7.15789474,  8.        ]),
                         'gamma': array([16.        , 11.95007169,  8.92526334,  6.66609605,  4.97877036,
        3.71854142,  2.77730229,  2.07431009,  1.54925964,  1.15711024,
        0.86422189,  0.6454696 ,  0.482088  ,  0.36006163,  0.26892265,
        0.20085281,  0.15001284,  0.11204151,  0.08368151,  0.0625    ])})

In [10]:
best_alpha_laplacian = cv.best_params_['alpha']
best_gamma_laplacian = cv.best_params_['gamma']
print(best_alpha_laplacian)
print(best_gamma_laplacian)

0.4210526315789469
0.2008528066417206


In [11]:
#MSE of all 3 modelsw

#Linear Regression
print(f'LINEAR REGRESSION TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, norm_train[:, -1])}')
print(f'LINEAR REGRESSION TEST MSE: {skl.metrics.mean_squared_error(predicted_test, norm_test[:, -1])}')

#RBF
RBFRegresor = kernel_ridge.KernelRidge(alpha = best_alpha_rbf, kernel = 'rbf', gamma = best_gamma_rbf)
RBFRegresor.fit(norm_train[:, :-1], norm_train[:, -1])
predicted_train = RBFRegresor.predict(norm_train[:, :-1])
print(f'GAUSSIAN KERNEL TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, norm_train[:, -1])}')
predicted_test = RBFRegresor.predict(norm_test[:, :-1])
print(f'GAUSSIAN KERNEL TEST MSE: {skl.metrics.mean_squared_error(predicted_test, norm_test[:, -1])}')

#Laplacian
LaplacianRegresor = kernel_ridge.KernelRidge(alpha = best_alpha_laplacian, kernel = 'laplacian', gamma = best_gamma_laplacian)
LaplacianRegresor.fit(norm_train[:, :-1], norm_train[:, -1])
predicted_train = LaplacianRegresor.predict(norm_train[:, :-1])
print(f'LAPLACIAN KERNEL TRAINING MSE: {skl.metrics.mean_squared_error(predicted_train, norm_train[:, -1])}')
predicted_test = LaplacianRegresor.predict(norm_test[:, :-1])
print(f'LAPLACIAN KERNEL TEST MSE: {skl.metrics.mean_squared_error(predicted_test, norm_test[:, -1])}')

LINEAR REGRESSION TRAINING MSE: 0.6278484956554817
LINEAR REGRESSION TEST MSE: 0.7021527395264012
GAUSSIAN KERNEL TRAINING MSE: 0.3988857186793212
GAUSSIAN KERNEL TEST MSE: 0.6259890718363516
LAPLACIAN KERNEL TRAINING MSE: 0.12760820922818175
LAPLACIAN KERNEL TEST MSE: 0.589727139061394


In [12]:
#Predict on new batch of wines 
path_to_new_wine = '/home/sathvikc/AMATH-482-2/AMATH-482/Homework Assignments/Homework 3/wine_new_batch.csv'
new_wine = genfromtxt(path_to_new_wine, delimiter=',')

#Linear Regression 
linear_predict = LinearRegressor.predict(new_wine)

#RBF
rbf_predict = RBFRegresor.predict(new_wine)

#Laplacian
laplacian_predict = LaplacianRegresor.predict(new_wine)

#print the predictions 
print(f'LINEAR PREDICTION: {linear_predict}')
print(f'RBF PREDICTION: {rbf_predict}')
print(f'LAPLACIAN PREDICTION: {laplacian_predict}')

#De-normalize the predictions
linear_predict = (linear_predict * train.std(axis=0)[-1]) + train.mean(axis=0)[-1]
rbf_predict = (rbf_predict * train.std(axis=0)[-1]) + train.mean(axis=0)[-1]
laplacian_predict = (laplacian_predict * train.std(axis=0)[-1]) + train.mean(axis=0)[-1]

LINEAR PREDICTION: [-12.08536286 -24.58319211 -15.8657664  -15.38470151 -13.24321852]
RBF PREDICTION: [3.79334664e-304 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000]
LAPLACIAN PREDICTION: [4.08511140e-13 2.10473821e-22 2.88448309e-15 1.92924723e-14
 3.30854929e-13]
